In [1]:
import os
os.chdir("../")
%pwd

'/Volumes/T7/DL_Skin_Cancer_Project/skin_cancer_diagnosis'

In [2]:
from pathlib import Path
import logging

Setting Environment Variables for ML flow

MLFLOW_TRACKING_URI=https://dagshub.com/uvaishnav/skin_cancer_diagnosis.mlflow \
MLFLOW_TRACKING_USERNAME=uvaishnav \
MLFLOW_TRACKING_PASSWORD=490c4cfe721d1b436d9c273a74eb000830a68aab \
python script.py

In [3]:
os.environ['MLFLOW_TRACKING_URI']= 'https://dagshub.com/uvaishnav/skin_cancer_diagnosis.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'uvaishnav'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '490c4cfe721d1b436d9c273a74eb000830a68aab'

Import Configurations

In [4]:
from skinCancerDiagnosis.entity.config_entity import ModelevalConfig
from skinCancerDiagnosis.config.configuration import ConfugarationManager
from skinCancerDiagnosis.components.data_prep import DataGenerator

Import Libraries

In [5]:
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from keras.models import load_model

In [6]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

Evaluation Component

In [7]:
class Evaluation:
    def __init__(self, config:ModelevalConfig) :
        self.config = config

    @staticmethod
    def load_model(path:Path)-> tf.keras.Model:
        return load_model(path)

    def evaluate_model(self,test_generator):
        self.model = self.load_model(self.config.model_path)

        # Get loss and acuracy
        self.score = self.model.evaluate(test_generator)
        
        # Generate predictions on the test data generator
        y_pred_prob = self.model.predict(test_generator)
        y_pred = np.argmax(y_pred_prob, axis=1)

        # Retrieve true labels from the test data generator
        y_true = test_generator.classes

        # Compute Micro-average Precision, Recall, and F1-Score
        self.micro_precision = precision_score(y_true, y_pred, average='micro')
        self.micro_recall = recall_score(y_true, y_pred, average='micro')
        self.micro_f1_score = f1_score(y_true, y_pred, average='micro')

    def log_into_mlflow(self,model_name):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {
                    "loss" : self.score[0],
                    "accuracy" : self.score[1],
                    "micro_precision" : self.micro_precision,
                    "micro_recall" : self.micro_recall,
                    "micro_f1" : self.micro_f1_score
                }
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                logging.info("regestiring Model to MLFlow")
                mlflow.keras.log_model(self.model, "model", registered_model_name=model_name)
            else:
                mlflow.keras.log_model(self.model, "model")
                logging.info("Inable to regestitor model to mlflow")

        


        

Model Evaluation Pipeline

In [8]:
try:
    config = ConfugarationManager()
    
    data_prep_config = config.get_data_prep_config()
    generator = DataGenerator(config=data_prep_config)
    test_generator = generator.get_test_generator()

    vgg_eval_config = config.get_evaluation_config(model_path='vgg19.h5')
    vgg_evaluator = Evaluation(vgg_eval_config)
    vgg_evaluator.evaluate_model(test_generator=test_generator)
    vgg_evaluator.log_into_mlflow(model_name="VGG19")
    
except Exception as e:
    raise e

[2024-04-23 23:13:21,591: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-23 23:13:21,593: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-23 23:13:21,594: INFO: common: created directory at: artifacts]
Found 1930 images belonging to 8 classes.


2024-04-23 23:13:21.788302: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-04-23 23:13:21.788321: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-04-23 23:13:21.788328: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-04-23 23:13:21.788345: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-23 23:13:21.788355: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[2024-04-23 23:13:22,182: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


2024-04-23 23:13:22.926293: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Volumes/T7/DL_Skin_Cancer_Project/canserenv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 42s 682ms/step - accuracy: 0.3469 - loss: 2.4684
61/61 ━━━━━━━━━━━━━━━━━━━━ 42s 682ms/step
[2024-04-23 23:14:49,451: INFO: 3261167456: regestiring Model to MLFlow]


2024/04/23 23:14:49 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
/Volumes/T7/DL_Skin_Cancer_Project/canserenv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG19'.
2024/04/23 23:15:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG19, version 1
Created version '1' of model 'VGG19'.
